# Initializations

In [9]:
from google.colab import drive
drive.mount("/content/drive/", force_remount=True)

Mounted at /content/drive/


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install mne
import mne
from mne import create_info
from mne.io import RawArray

# Loading Data

In [ ]:
#### Load data ####
raw_data = {"Victor":[],"Aaron":[], "Ben":[], "Thu":[], "Nat":[] }

  # Iterate thru all trials for each subject
for subj in raw_data.keys():
      # Load data from CSV into an array
      path = '/content/drive/MyDrive/EEG-Fear-Detection'+'/OpenBCISession_'+subj+'/' + subj+'.txt'
      # Skip's first five lines since it's text
      df = pd.read_csv(path, delimiter=',',skiprows=5)

      trial_data = df.to_numpy()

      # Declares channel names and types of each set of data
      sfreq = 250  # sample rate in Hz
      trial_data = trial_data[:, :-1]
      ch_names = ['Channel {}'.format(i) for i in range(trial_data.shape[1])]
      ch_types = ['eeg' for i in range(trial_data.shape[1])]

      # Create info structures and RawArray objects for each set of data
      info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)
      raw = mne.io.RawArray(trial_data.T, info)

      # Removing irrelevant channels
      ch_names = [raw.ch_names]
      ch_names_to_keep = [ch_names[0][1:9]]
      raw = raw.pick_channels(ch_names_to_keep[0])

      # Add RawArray
      raw_data[subj].append(raw)

# Filtering and Segmenting Data

In [ ]:
#### Truncate and filter data ####
data_segments = pd.read_csv('/content/drive/MyDrive/EEG-Fear-Detection/EEG_Data_Segmentation.csv')

filtered_data = {"Victor":[], "Aaron":[], "Ben":[], "Thu":[], "Nat":[] }
# variable to hold all different segment marker names
segments = [["R1-State Start","R1-State End"],["S1-State Start","S1-State End"],["R2-State Start","R2-State End"],["S2-State Start","S2-State End"],["R3-State Start","R3-State End"],["S3-State Start","S3-State End"]]

# Iterate thru all trials for each subject
for subj in filtered_data.keys():
    # set variable to the eeg object
    curr_trial = raw_data[subj][0]
    print(curr_trial)
    # Filter current trial data
    filtered_trial = curr_trial.copy().filter(l_freq=0.5, h_freq=45, picks=None,
                                 method='iir', l_trans_bandwidth='auto',
                                 h_trans_bandwidth='auto', filter_length='auto', phase='zero')

    # Crop filtered_trial to within experiment duration
    exp_info = data_segments[data_segments["Participant"] == subj].index
    # Segment filtered_trial into different sections based on segment marker names
    for labels in segments :
      filtered_section = filtered_trial.copy().crop(tmin=data_segments.at[exp_info[0], labels[0]],tmax=data_segments.at[exp_info[0], labels[1]])
      filtered_data[subj].append(filtered_section)


# Normalizing Data Length

In [13]:
for subj in filtered_data:
  for i in range(2):
    # Find which trial of same state has minimum length
    min_len = np.argmin([len(filtered_data[subj][i]), len(filtered_data[subj][i+2]), len(filtered_data[subj][i+4])])
    segs = [filtered_data[subj][i], filtered_data[subj][i+2], filtered_data[subj][i+4]]
    # Create start time and end time based of minimum-length trial
    start_time = filtered_data[subj][i*2].times[0]
    end_time = start_time + (segs[min_len].times[-1] - segs[min_len].times[0])
    for trials in segs:
      # Crop each section using the new start and end time
      trials = trials.crop(tmin=start_time, tmax=end_time)

# Averaging Similar State Trials

In [ ]:
agr_data = {"Victor":[], "Aaron":[], "Ben":[], "Thu":[], "Nat":[] }
avg_data = {"Victor":[[],[]], "Aaron":[[],[]], "Ben":[[],[]], "Thu":[[],[]], "Nat":[[],[]] }

info = mne.create_info(ch_names=ch_names_to_keep[0], sfreq=sfreq, ch_types=ch_types[0:8])

for subj in filtered_data:
  # Loop twice for each state, and eight times for each channel
  for i in range(2):
    for j in range(8):
      # Create an array of arrays of same state trials, and calculate mean
      mean_array = np.array([filtered_data[subj][i][j][0], filtered_data[subj][i+2][j][0], filtered_data[subj][i+4][j][0]])
      x = np.mean(mean_array, axis=0)
      avg_data[subj][i].append(x)

    avg_data[subj][i] = np.array(avg_data[subj][i])
    # Turn averaged numpy arrays back into RawArray
    x = mne.io.RawArray(avg_data[subj][i].squeeze(axis=1), info)
    agr_data[subj].append(x)


# Seperating Trials By Bands

In [ ]:
#### Filter Data by Bands ####
band_data = {"Victor":[], "Aaron":[], "Ben":[], "Thu":[], "Nat":[] }


# Iterate thru all trials for each subject,
# Two trials for each subject, one trial for each state
for subj in band_data.keys():
  for i in range(2):
    # set variable to the eeg object
    curr_trial = agr_data[subj][i]
    # Filter pre-processed trial data by band frequencyv

    beta_waves = curr_trial.copy().filter(l_freq=12.5, h_freq=39.5, picks=None,
                                     method='iir', l_trans_bandwidth='auto',
                                     h_trans_bandwidth='auto', filter_length='auto', phase='zero')

    theta_waves = curr_trial.copy().filter(l_freq=4, h_freq=7.5, picks=None,
                                     method='iir', l_trans_bandwidth='auto',
                                     h_trans_bandwidth='auto', filter_length='auto', phase='zero')

    delta_waves = curr_trial.copy().filter(l_freq=0.5, h_freq=5, picks=None,
                                     method='iir', l_trans_bandwidth='auto',
                                     h_trans_bandwidth='auto', filter_length='auto', phase='zero')

    # Add band-passed data to band_data dictionary
    band_data[subj].append(beta_waves)
    band_data[subj].append(theta_waves)
    band_data[subj].append(delta_waves)

# Data Visualization - PSD Graphs

In [ ]:
# Frequencies according to Beta, Theta, and Delta in that order
band_freqs = [[12.5,39.5],[4,7.5],[0.5,5]]

# plot three band graphs per state per person
for subj in band_data:
  print(subj, "\n")
  for i in range(2):
    for j in range(3):
      band_data[subj][i].compute_psd(method="welch", verbose=False,fmax=band_freqs[j][1]).plot(average=True, ci=None)